In [111]:
from sympy import *
import numpy as np 
init_printing()

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

D11 = 2*eta; D12 = 0*eta; D13 = 0*eta;
D21 = 0*eta; D22 = 2*eta; D23 = 0*eta;
D31 = 0*eta; D32 = 0*eta; D33 = 1*eta;

Dv = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
Ed = Matrix( [ [Exx], [Ezz], [Gxz] ] )
T  = Dv*Ed;
for i in range(len(stress_labels)):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )


Txx = 2*Exx*eta;
Tzz = 2*Ezz*eta;
Txz = Gxz*eta;


In [107]:
# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])
#----------------------------------------#
ExxW = 1/dx*(uC-uW);
EzzW = 0;
GxzW = 0
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs(eta,'etaW')
#TxxW = 2* etaW* ExxW
#----------------------------------------#
ExxE = 1/dx*(uE-uC)
EzzE = 0;
GxzE = 0
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs(eta,'etaE')
#----------------------------------------#
ExxS = 0
EzzS = 0
GxzS = 1/dz*(uC-uS) + 1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs(eta,'etaS')
#----------------------------------------#
ExxN = 0
EzzN = 0
GxzN = 1/dz*(uN-uC) + 1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs(eta,'etaN')
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
Fx = -Fx
for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = -2*etaW/pow(dx, 2);
uC = -(-etaN/dz - etaS/dz)/dz - (-2*etaE/dx - 2*etaW/dx)/dx;
uE = -2*etaE/pow(dx, 2);
uS = -etaS/pow(dz, 2);
uN = -etaN/pow(dz, 2);
vSW = -etaS/(dx*dz);
vSE = etaS/(dx*dz);
vNW = etaN/(dx*dz);
vNE = -etaN/(dx*dz);


In [90]:
# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])
#----------------------------------------#
ExxS = 0
EzzS = 1/dz*(vC-vS);
GxzS = 0
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs(eta,'etaS')
#----------------------------------------#
ExxN = 0
EzzN = 1/dz*(vN-vC);
GxzN = 0
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs(eta,'etaN')
#----------------------------------------#
ExxW = 0
EzzW = 0
GxzW = 1/dz*(uNW-uSW) + 1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs(eta,'etaW')
# #----------------------------------------#
ExxE = 0
EzzE = 0
GxzE = 1/dz*(uNE-uSE) + 1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs(eta,'etaE')
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(9):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = -etaW/pow(dx, 2);
vC = -(-2*etaN/dz - 2*etaS/dz)/dz - (-etaE/dx - etaW/dx)/dx;
vE = -etaE/pow(dx, 2);
vS = -2*etaS/pow(dz, 2);
vN = -2*etaN/pow(dz, 2);
uSW = -etaW/(dx*dz);
uSE = etaE/(dx*dz);
uNW = etaW/(dx*dz);
uNE = -etaE/(dx*dz);
